In [49]:
%pylab inline

from library import lasagnennet as LN
from library import features as F
from library import scale as S
import numpy as np
import theano as TH
import pandas as pa
import lasagne as L

Populating the interactive namespace from numpy and matplotlib


In [52]:
reload(S)
S.scale()

['ID', 'TIME', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'ICU']
['ID', 'TIME', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12', 'L13', 'L14', 'L15', 'L16', 'L17', 'L18', 'L19', 'L20', 'L21', 'L22', 'L23', 'L24', 'L25']
['ID', 'AGE']
Column :  V1 max Value :  290.0 mean Value :  122.710736712 Mean by Max ratio :  0.42314047142
Column :  V1 Mean by Max ratio :  0.42314047142
Column :  V2 max Value :  200.0 mean Value :  69.2100425966 Mean by Max ratio :  0.346050212983
Column :  V2 Mean by Max ratio :  0.346050212983
Column :  V3 max Value :  219.0 mean Value :  86.8153202933 Mean by Max ratio :  0.39641698764
Column :  V3 Mean by Max ratio :  0.39641698764
Column :  V4 max Value :  82.0 mean Value :  20.4260971774 Mean by Max ratio :  0.249098746066
Column :  V4 Mean by Max ratio :  0.249098746066
Column :  V5 max Value :  102.0 mean Value :  97.6664309932 Mean by Max ratio :  0.957514029345
Column :  V5 Mean by Max ratio :  0.957514029345
Column :  V6 max Value :  1

In [72]:
reload(F)
F.getfeatures(
    vital_file='Training_Dataset/vital_train.csv',
    lab_file='Training_Dataset/lab_train.csv',
    age_file='Training_Dataset/age_train.csv',
    label_file='Training_Dataset/id_label_train.csv',
    prefix=''
)

Getting features...


KeyboardInterrupt: 

In [55]:
train_feats = pa.read_csv('train_feats.csv')
train_targets = pa.read_csv('train_targets.csv')
val_feats = pa.read_csv('validation_feats.csv')
val_targets = pa.read_csv('validation_targets.csv')

In [56]:
print train_targets.columns

Index([u'ID', u'TIME', u'LABEL', u'ICU'], dtype='object')


In [57]:
tx = np.asarray(train_feats, dtype=TH.config.floatX)
tyall = np.asarray(train_targets, dtype=TH.config.floatX)
ty = np.asarray(train_targets['LABEL'], dtype=TH.config.floatX).reshape(-1, 1)
vx = np.asarray(val_feats, dtype=TH.config.floatX)
vyall = np.asarray(val_targets, dtype=TH.config.floatX)
vy = np.asarray(val_targets['LABEL'], dtype=TH.config.floatX).reshape(-1, 1)

In [58]:
print tx.shape, tyall.shape, ty.shape, vx.shape, vyall.shape, vy.shape

(471667, 641) (471667, 4) (471667, 1) (156770, 641) (156770, 4) (156770, 1)


In [70]:
reload(LN)
nneta = LN.nnet(n_in=641, n_out=1, h_layers=[1000,1000],
                i_drop=None, l_drops=None, lam=0.0,
                nonlinearity=L.nonlinearities.sigmoid
           )

In [71]:
nneta.train(x=tx, y=ty, lrate=1, gamma=0.9, batch_size=1000, iters=100,
            testx=vx, testy=vy, filename='model.npz'
           )

Training ... 
Iteration 0, Loss = 4.4704550813
Testing ...
Mean accuracy = 0.0
Iteration 1, Loss = 2.05406160235
Iteration 2, Loss = 1.46645686491
Iteration 3, Loss = 0.840035182691
Iteration 4, Loss = 0.786489365821
Iteration 5, Loss = 0.786481411628
Iteration 6, Loss = 0.786472140993
Iteration 7, Loss = 0.786464402403
Iteration 8, Loss = 0.786458370537
Iteration 9, Loss = 0.786453036368
Iteration 10, Loss = 0.786448141692
Testing ...
Mean accuracy = 0.0
Iteration 11, Loss = 0.786443763694
Iteration 12, Loss = 0.786439187905
Iteration 13, Loss = 0.78643509964
Iteration 14, Loss = 0.786430462473
Iteration 15, Loss = 0.786425634344
Iteration 16, Loss = 0.786421017751
Iteration 17, Loss = 0.786416512807
Iteration 18, Loss = 0.786411605584
Iteration 19, Loss = 0.786406716285
Iteration 20, Loss = 0.786401201761
Testing ...
Mean accuracy = 0.0


KeyboardInterrupt: 

In [42]:
print nneta.tester(vx[np.where(vy==0)[0]])
print vx[np.where(vy==0)[0]].shape

[array([[ 1.],
       [ 1.],
       [ 1.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32)]
(138276, 641)
